In [ ]:
import os

import geopandas as gpd
import pandas as pd
from odc.geo import BoundingBox

from coastlines.grids import PHILIPPINES_25, PHILIPPINES_CRS

os.environ["USE_PYGEOS"] = "0"

In [ ]:
# Select relevant tiles and convert to Geopandas
bounds = BoundingBox(115, 3, 130, 23, crs="EPSG:4326").to_crs(PHILIPPINES_CRS)
geom = PHILIPPINES_25.geojson(bbox=bounds)
tiles_projected = gpd.GeoDataFrame.from_features(geom, crs="EPSG:4326").to_crs(
    PHILIPPINES_CRS
)

tiles_projected.explore()

In [ ]:
# Get PH boundaries from humdata
# https://data.humdata.org/dataset/cod-ab-phl
url = "https://s3.ap-southeast-2.amazonaws.com/files.auspatious.com/coastlines/ph_boundaries.fgb"

phl_adm = gpd.read_file(url)

In [ ]:
ph_simple = phl_adm.simplify(0.001)

ph_simple.explore()

In [ ]:
# Prepare a dataset that contains continental coastlines with a 15 km buffer
area_limit = 1e9
ph = ph_simple.to_crs(PHILIPPINES_CRS)
small_islands = ph[ph.geometry.area < area_limit]
large_islands = ph[ph.geometry.area >= area_limit]

# Buffer the long coastlines boundaries
coast = large_islands.boundary.buffer(15_000)
# And the small islands
islands = small_islands.buffer(15_000)

# Add some extra reefs and islands
points = [
    [120.4158462, 12.6594815],  # Apo Reef
    [119.7286163, 5.8124749],  # Sitio Tahao
    [119.8205949, 8.8059727],  # Tubbataha Reefs
    [118.5168170, 7.7678431],  # Bancoran Island
]

points = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy([p[0] for p in points], [p[1] for p in points]),
    crs="EPSG:4326",
)
points = points.to_crs(PHILIPPINES_CRS).buffer(15_000)

# Now merge them together
buffered = pd.concat([coast, islands, points])

aoi = gpd.GeoDataFrame(buffered, geometry=buffered.geometry).dissolve()
aoi.explore()

In [ ]:
# Clip tiles to AOI
coastal_tiles = tiles_projected.clip(aoi)

In [ ]:
# Dump a simple list of all tile_ids in a text file
coastal_tiles["idx"].to_csv(
    "../data/raw/philippines_tile_ids.txt", index=False, header=False
)

In [ ]:
# These tiles can be removed safely
SKIP_TILES = [
    # Edges and tiny areas
    "207,212",
    "196,219",
]

coastal_tiles_filtered = coastal_tiles[~coastal_tiles["idx"].isin(SKIP_TILES)]
coastal_tiles_filtered.explore()

In [ ]:
# Export the results as GeoJSON
coastal_tiles_filtered.rename(columns={"idx": "id"}, inplace=True)
coastal_tiles_filtered.to_crs("EPSG:4326").to_file(
    "../data/raw/philippines_tiles.geojson", driver="GeoJSON"
)